Import Dependecies

In [1]:
import cvxpy as cp
import numpy as np
import math
A = np.array([[ 1, 1, 0, 0, 0],
[ 0, -1, 1, 1, 0],
[ 0, 0, 0, -1, 1],
[-1, 0, -1, 0, -1]], dtype=np.float64)
A = A[0:-1,:]
R1 = 1.
R2 = 1e3
R5 = 1e2
VT = 26
IS = 1
VS = 1e3

Solve the problem

In [2]:
i = cp.Variable(5)
OBJ1 = VS*i[0] + (1./2)*R1*cp.square(i[0])
OBJ2 = (1./2)*R2*cp.square(i[1])
OBJ3 = VT*IS*(-cp.entr(1. + i[2]/IS) - i[2]/IS)
OBJ4 = VT*IS*(-cp.entr(1. + i[3]/IS) - i[3]/IS)
OBJ5 = (1./2)*R5*cp.square(i[4])
obj = cp.Minimize(OBJ1 + OBJ2 + OBJ3 + OBJ4 + OBJ5)
constr = [A * i == 0.]
problem = cp.Problem(obj, constr)
#problem.solve(verbose=True, solver=SCS, eps=1e-4)
problem.solve(verbose=True)
e = -problem.constraints[0].dual_value
v = np.array([[VS + R1*float(i.value[0])],
[R2*float(i.value[1])],
[VT*math.log(1. + float(i.value[2])/IS)],
[VT*math.log(1. + float(i.value[3])/IS)],
[R5*float(i.value[4])]])
v_err = v - np.transpose(A) * e
rel_err = np.linalg.norm(v_err) / np.linalg.norm(v)

                                     CVXPY                                     
                                     v1.3.1                                    
(CVXPY) May 14 01:55:04 PM: Your problem has 5 variables, 1 constraints, and 0 parameters.
(CVXPY) May 14 01:55:04 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 14 01:55:04 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 14 01:55:04 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 14 01:55:04 PM: Compiling problem (target solver=ECOS).
(CVXPY) May 14 01:55:04 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> 

/usr/local/lib/python3.10/dist-packages/cvxpy/expressions/expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 1 times so far.

  warnings.warn(msg, UserWarning)


Relative error

In [3]:
print("Relative error in KVL: %e\n" % rel_err)

Relative error in KVL: 1.420047e+00



Voltages

In [4]:
print(v)

[[999.01703386]
 [982.96613594]
 [ 16.05094564]
 [  3.15391948]
 [ 12.89687387]]


Currents

In [5]:
print(i.value)

[-0.98296614  0.98296614  0.8539974   0.12896874  0.12896874]


potentials

In [7]:
print(e)

[999.017032    16.05093715  12.89702156]


In [12]:
print((A.T@e))

[999.017032   982.96609486  16.05093715   3.15391558  12.89702156]
